# Fluent Python by Luciano Ramalho Notes

## Chapter 2: An Array of Sequences


In [ ]:
# Generator Expressions (genexp) -> Basically list comp but for other data structs ie...
color_size = tuple((color, size) for color in ["red", "white", "blue"] for size in ["Large", "Medium"])
print(color_size)  # -> tuple
# The double "for" works for list comp as well. Analogous to...
# for color in ...:
    # for size in ...:
        # ...
     
# Genexps can also be used as iterators. The benefit is that the list is never written into memory!
for tshirt in tuple((color, size) for color in ["red", "white", "blue"] for size in ["Large", "Medium"]):
    print(tshirt) 

In [ ]:
# A "*" in tuple unpacking takes all remaining values and stores it in the variable
first, second, *rest = (1,2,3,4)
print(first, second, rest)

In [ ]:
# collections.namedtuple are just like tuple but allow you to name the parameters.
from collections import namedtuple
City = namedtuple('city', 'name country population')
tokyo = City('tokyo', 'japan', 36)
print(tokyo[0], tokyo.country)

In [ ]:
s = (3,4)
s += (4,5)
print(s)

In [ ]:
# Indexing with "var[:::]" creates a slicing object which is used to index. We can also slice directly by self making a "slice object"
# and can thus name slices
my_list = ['red', 'white', 'apple']
colors = slice(0, 2, 1)
print(my_list[colors])

# ... refers to all remaining dims in ndarray
import numpy as np
mat = np.zeros((2,2,2,2))
print(mat[0, ...])

In [ ]:
# array.array is basically a C++ array and is much more efficient than a python list. It can only contain numbers...
# 'd' -> double precision float... 'b' is a byte. I'm sure they have others...
from array import array
my_arr = array('d', [2,2])
print(my_arr, my_arr[0])
# method my_arr.fromfile(open('filename', open_type)) takes numbers from file and stores it in array. my_arr.tofile() puts item in the file

## Chapter 3: Dictionaries and Sets


In [ ]:
# All immutable types are "hashable." This means strings, bytes, numeric types and even tuples are even hashable. 
# Tuples are only hashable if all elements inside are also hashable. i.e. tuple of list is unhashable but tuple-ception is fine
# A "frozenset" takes a mutable object and makes it immutable. Thus, while a list is mutable, the frozenset of a list is immutable
print(hash((3,4)))
print(hash("hello"))
print(hash(frozenset([3,4])))
print(hash([3,4]))

In [ ]:
a = dict(one=1, two=2)
b = {'one':1, 'two':2}
c = dict(zip(['one', 'two'], [1, 2]))
d = dict([('two', 2), ('one', 1)])

nums = [('one', 1), ('two', 2)]
e = {string: num for string, num in nums}    # Dict comprehension

a == b == c == d == e

In [ ]:
# Set default is like dict.get() but actually applies the key value pair to the dict
my_dict = {'five': 5, 'two':2}
print(my_dict)
my_dict.setdefault('six', 6)
print(my_dict)
my_dict.setdefault('five', 5)
print(my_dict)

In [ ]:
# A lot going on here... collections.defaultdict(object_type) automatically calls my_dict.setdefault() when using d[] and instantiates an
# empty object of type object_type. The dunder-repr here just allows nice print outs 
class random_class:
    rand_int: int = 0
    def __repr__(self) -> str:
        return f"my rand int {self.rand_int}"

from collections import defaultdict
d = defaultdict(random_class)
print(d["hi"])
d["hello"] = 14
print(d)

In [ ]:
# collections.OrderedDict -> dictionaries that retain insertion order
# collections.ChainMap -> takes in a bunch of maps and searches through all (in order) until it finds key
# collections.UserDict -> Is a base class for user custom dictionaries/maps
# collecitons.Counter -> stores the number of value references for a given key... cool example below...
from collections import Counter
ct = Counter('asah edfjhdsgkj asg aslkjKHTy')
print(ct)
print(ct.most_common(3))


In [ ]:
# A types.MappingProxyType(map) takes in a map and spits out a read-only (immutable) map view. If the original dictionary changes, 
# the proxy changes as well. However, one cannot change the proxy
d = Counter('asah edfjhdsgkj asg aslkjKHTy')
from types import MappingProxyType
d_proxy = MappingProxyType(d)
print(d_proxy['a'])
d.update('aaa')
print(d_proxy['a'])
try:
    d_proxy['a'] = 4
except Exception as e:
    print(e)


In [ ]:
# A set is a collection of unique objects. Set removes duplicates and has operators to go with it.
l = set(['hello', 'hi', 'hi']) 
print(l)
t = {'hi', 'jello'} # {} means dict but if it's initialized without key-pairs then it becomes a set.
print(l & t)
print(l | t)
print(l - t)

from unicodedata import name
cool_set = {chr(i) for i in range(32,256) if 'SIGN' in name(chr(i), '')}
print(cool_set)

## Chapter 4: Text Vs. Byte

## Chapter 5: Functions as Objects

In [ ]:
import random
class Deck:
    def __init__(self, items) -> None:
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty deck')
    
    # This dunder method allows class instances to work as functions  
    def __call__(self):
        return self.pick()
    
my_deck = Deck(range(10))
card = my_deck()
print(f"My card is: {card}")
print(f"Callable? {callable(my_deck)}")

In [ ]:
# Operator library has a bunch of standard operation functions (i.e. add, sub, concat... mul in this case is a function that takes two parameters and 
# multiplies them). Operator also has "methodcaller" which allows you to call object methods as well as itemgetter and attrgetter gets attributes you may need.mul
# This is helpful for functions such as sorted. In sorted, we can set a key to a specific attribute
from operator import mul

# Partial takes a function and some parameters and returns the func with fewer parameters. See how it's used below
from functools import partial

triple = partial(mul, 3)
print(triple(6))

## Chapter 6: Design Patterns with First-Class Functions

In [ ]:
class Order:
    
    def __init__(self, cart) -> None:
        self.cart = cart
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total for item in self.cart)
        return self.__total

In [ ]:
# Searches for all global variables that end with "_promo"
promos = [globals()[name] for name in globals() if name.endswith('_promo')]

# OR, we can have a module with a bunch of these functions and use...
import inspect
import promotions
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]


## Chapter 7: Function Decorators and Closures

In [ ]:
# This example display interesting python behaviour. If a variable is assigned within a function, python assumes that this variable must be a local
# var during compilation of the function. Therefore, in f2, even though b is technically defined just as it was in f1, python throws an error
# that b does not exist. "global" keyword overrides this behaviour
b = 2
def f1(a):
    print("F1")
    print(a)
    print(b)

def f1_5(a):
    print("\nF1.5")
    global b
    print(a)
    print(b)
    b = 4

def f2(a):
    print("\nF2")
    print(a)
    print(b)
    b = 4
    
f1(1)
f1_5(1)
f2(1)

In [ ]:
# Let's say we want a function that calculates the running average of numbers

# We can use classes...
class Averager():
    def __init__(self) -> None:
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
# Or we can use nested functions and python's implementation of "closures"...
def make_averager():        # The contents of this function are called the "closure"
    series = []             # -> Free var. Closure is a func that retains bindings of the free vars when called later.
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

avg_class = Averager()
avg_func = make_averager()

print("Avg of [10]...")
print(avg_class(10))
print(avg_func(10))

print("\nAvg of [10, 11]...")
print(avg_class(11))
print(avg_func(11))

print("\nAvg of [10, 11, 12]...")
print(avg_class(12))
print(avg_func(12))

In [ ]:
# The above code is inefficient because we store every value in a list. A better implementation is shown below.
def make_averager_best():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total           # Because "count += 1" is really "count = count + 1", we require this "nonlocal" term
        count += 1
        total += new_value
        return total/count
    
    return averager

# The nonlocal term prevents the behaviour from 2 blocks above where an "=" sign implies that the variable is local. "nonlocal" is similar to global
# but we don't want to imply that those two values are global either. Therefore, the "nonlocal" keyword was invented for this reason.

avg_func = make_averager_best()

print("Avg of [10]...")
print(avg_func(10))

print("\nAvg of [10, 11]...")
print(avg_func(11))

print("\nAvg of [10, 11, 12]...")
print(avg_func(12))

In [ ]:
def d1(func):
    print("Execute d1")
    return func

def d2(func):
    print("Execute d2")
    return func

@d1
@d2
def f():
    print("Calling f")
    
f()
d1(d2(f))()

In [ ]:
# Decorator factories allow decorators to take parameters by returning decorators
# Let's say we want to time all functions but only want print outs for some.

import time

# Decorator factory
def timer(print_=False):
    # Decorator
    def clock(func):
        # Decorator function
        def clocked(*args):
            t0 = time.perf_counter()
            result = func(*args)
            elapsed = time.perf_counter() - t0
            if print_:
                name = func.__name__
                arg_str = ', '.join(repr(arg) for arg in args)
                print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
            return result
        return clocked
    return clock

@timer(print_=True)
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

@timer()
def double(n):
    return n*2

factorial(10)
double(10)

## Chapter 8: Object-Oriented Idioms

In [ ]:
# Variables in python are sticky notes over boxes NOT the boxes themselves as shown in the example below
a = [1,2,3]
b = a
b.append(4)
a

In [ ]:
# "==" compares the values of 2 objects while "is" compares the object ids i.e. whether they are aliases of the same instance
# "is" is faster than "==" 
l1 = [4, 5, 6]
l2 = l1

l3 = [4, 5, 6]

print(f"l1 == l2: {l1 == l2}")
print(f"l1 is l2: {l1 is l2}")

print(f"l1 == l3: {l1 == l3}")
print(f"l1 is l3: {l1 is l3}")

In [ ]:
# Easiest way to copy
l1 = [[1], 2, 3, 4]
l2 = l1             # Alias i.e. same object

# The following are shallow copies. The inner values are references to the original objects in the list so only the outter container
# is a copy. If the values are immutable (i.e. tuples, numbers, etc.), this should not matter. But is affected if a value is a list or other mutablae
# sequences! 
l3 = list(l1)       # Shallow copy
l4 = l1[:]          # Shallow copy

print(f"l1 == l2: {l1 == l2}")
print(f"l1 is l2: {l1 is l2}")

print(f"l1 == l3: {l1 == l3}")
print(f"l1 is l3: {l1 is l3}")

print(f"l1 == l4: {l1 == l4}")
print(f"l1 is l4: {l1 is l4}")

print(f"l1: {l1}")
l4[0].append(5)
l4[1] = 5
l1.append(20)
print(f"l1: {l1}")
print(f"l4: {l4}")

In [ ]:
import copy

l1 = [[1], 2, 3, 4]
l2 = copy.copy(l1)          # Shallow copy
l3 = copy.deepcopy(l1)      # Deep copy

l1[0].append(3)
print("l2:", l2)
print("l3:", l3)

In [ ]:
# Parameters are sent in as references but like shallow copies, immutable paramers will not be changed. 
def f(a, b):
    a += b
    return a

x, y = 1, 2
print(f"f({x}, {y}):", f(x, y), "-> x, y =", (x, y))

x, y = [1,3], [2,4]
print(f"f({x}, {y}):", f(x, y), "-> x, y =", (x, y))

In [ ]:
# Dangers of mutable default parameters
class Bus():
    def __init__(self, passengers=[]) -> None:
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        
    def __repr__(self) -> str:
        return str(self.passengers)
        
# Our passenger list gets assigned to the default parameter. Then we append to that original list
bus1 = Bus()
bus1.pick("George")

# When a second bus is initialized, it now gets assigned to the altered list!
bus2 = Bus()
print("bus2:", bus2)
print(Bus.__init__.__defaults__)

# Similarly...
basketball_team = ["Lebron", "Dwade", "Dinosaur"]
baller_bus = Bus(basketball_team)
baller_bus.drop("Lebron")
print(f"basketball team", basketball_team)

In [ ]:
# Better version of above
class Bus():
    def __init__(self, passengers=None) -> None:
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        
    def __repr__(self) -> str:
        return str(self.passengers)
        
# Our passenger list gets assigned to the default parameter. Then we append to that original list
bus1 = Bus()
bus1.pick("George")

# When a second bus is initialized, it now gets assigned to the altered list!
bus2 = Bus()
print("bus2:", bus2)
print(Bus.__init__.__defaults__)

# Similarly...
basketball_team = ["Lebron", "Dwade", "Dinosaur"]
baller_bus = Bus(basketball_team)
baller_bus.drop("Lebron")
print(f"basketball team", basketball_team)

## Chapter 9: A Pythonic Object

String representation of objects:  
  * repr() -> How a developer wants to see it  
  * str()  -> How a user wants to see it  

In [ ]:
# Vector class in python with a lot of useful dundermethod implementations
class Vector2d:
    typecode = 'd' # Class attribute we'll use when converting instances to/from bytes
    
    # Generally attrs are stored in dicts but dicts have very significant memory overhead. __slots__ tells the interpreter exactly what's being stored
    # and stores it as a tuple instead, cutting the memory usage significantly
    __slots__ = ('__x', '__y') 
    
    def __init__(self, x, y) -> None:
        self.__x = float(x)         # Read-only vars
        self.__y = float(y)
     
    # This decorator marks the function as a getter property
    @property    
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
        
    def __iter__(self):
        return (i for i in (self.x, self.y))    # Allows tuple unpacking i.e. x,y = vector_2d will work
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self) # Uses the above iter to print out the x,y values
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))) # Don't understand what this does
    
    def __eq__(self, o: object) -> bool: # Issue! If another objects spits out an equivalent tuple, then this will trigger equality!
        return tuple(self) == tuple(o)
    
    # To have hash method, we need vector to be immutable. So, x and y are read-only params. We also need an __eq__ method.
    def __hash__(self) -> int:
        # We want the hash to depend on x and y so that two defined vectors with the same x,y have the same hash. ^ is a bitwise XOR
        # and is suggested for mixing hashes. 
        return hash(self.x) ^ hash(self.y)
    
    def __abs__(self):
        from math import hypot
        return hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    # Creates instance from bytes directly
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    # Class method vs. static method. Class method is a method that belongs to a class and takes in the class as its first argument rather 
    # than an instance (i.e. self). These are mostly used as alternate constructors. Here, we create an alternate constructor spinning up
    # a class from bytes. A static method (or @staticmethod) is a function that takes in no extra parameters and is more like a function that happens to be 
    # defined in a class
    
    def angle(self):
        from math import atan2
        return atan2(self.y, self.x)
    
    def __format__(self, format_spec: str='') -> str:
        if format_spec.endswith('p'):
            format_spec = format_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
            
        components = (format(c, format_spec) for c in coords)
        return outer_fmt.format(*components)
    
vector = Vector2d(3,4)
print("Calling polar format 3 dec places", format(vector, '.3ep'))
print("Calling normal format 3 dec places", format(vector, '.3e'))
print("Calling str representation", vector)
print("Calling repr representation", repr(vector))

x, y = vector
print("Tuple unpacking repr", (x, y))
print("Obj equality", Vector2d(3,4) == vector)
my_dict = {vector: "hello", Vector2d(5,2): 'hi'}
print("Since hashable, we can use in dict keys", my_dict)

v1 = eval(repr(vector)) # Evaluates the string repr of vector, thereby creating an identical obj
print("Equivalent v1:", v1, v1 == vector)

# If an object var starts with "__", python auto concatenates _{class_name} to it as a simple protection.
# This prevents unknowing users to change variable that they shouldn't be changing but doesn't protect from malicious use!
# This is called name mangling
try:
    print(vector.__x) 
except Exception as e:
    print(e)
    
print(vector._Vector2d__x)